In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import re

import os
import matplotlib.pyplot as plt
import scienceplots

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

plt.style.use('nature')
plt.rcParams['legend.frameon'] = False
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

## 01. Init

In [2]:
root_dir = '../'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

In [4]:
input_dir = f"{cfg.config['out_dir']}/out-publish-normalized"
out_dir = f"{cfg.config['out_dir']}/stats"

out_figure_dir = f'{out_dir}/figure'
os.makedirs(out_figure_dir, exist_ok=True)

data_dict_filename = f"{root_dir}/{COMBINE_harmonizer.DATA_DICTIONARY_EXCEL}"

In [5]:
COMBINE_harmonizer.init_inv_mapping(data_dict_filename)

_INV_VALUE_MAP_MAIN = COMBINE_harmonizer.build_inv_value_map(data_dict_filename, COMBINE_harmonizer.SHEET_MAIN)
_INV_VALUE_MAP_FOLLOWUP = COMBINE_harmonizer.build_inv_value_map(data_dict_filename, COMBINE_harmonizer.SHEET_FOLLOW_UP)
_INV_VALUE_MAP_DERIVED_DATA = COMBINE_harmonizer.build_inv_value_map(data_dict_filename, COMBINE_harmonizer.SHEET_DERIVED_DATA)

build_inv_value_map: (0/1064) variable: center type: center
[INFO] _build_inv_value_map: to inv-text: type: center
build_inv_value_map: (1/1064) variable: subjectID type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (2/1064) variable: siteID type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (3/1064) variable: birthDate type: date
[INFO] _build_inv_value_map: to inv-text: type: date
build_inv_value_map: (4/1064) variable: birthNumber type: int
build_inv_value_map: (5/1064) variable: screenComment type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (6/1064) variable: coreTempLess32p5COverEq2Hr_e type: bool
build_inv_value_map: (7/1064) variable: coreTempLess33p5COver1Hr_e type: bool
build_inv_value_map: (8/1064) variable: coreTempLess34COver1Hr_e type: bool
build_inv_value_map: (9/1064) variable: first6HrCoolByClinicalProtocol_e type: bool
build_inv_value_map: (10/1064) variable: chromos

build_inv_value_map: (0/355) variable: followupCenter type: center
[INFO] _build_inv_value_map: to inv-text: type: center
build_inv_value_map: (1/355) variable: siteID type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (2/355) variable: birthDate type: date
[INFO] _build_inv_value_map: to inv-text: type: date
build_inv_value_map: (3/355) variable: visitDate type: date
[INFO] _build_inv_value_map: to inv-text: type: date
build_inv_value_map: (4/355) variable: birthNumber type: int
build_inv_value_map: (5/355) variable: center type: center
[INFO] _build_inv_value_map: to inv-text: type: center
build_inv_value_map: (6/355) variable: subjectID type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (7/355) variable: followupID type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (8/355) variable: center_orig type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (9/3

build_inv_value_map: (0/148) variable: center type: center
[INFO] _build_inv_value_map: to inv-text: type: center
build_inv_value_map: (1/148) variable: followupCenter type: center
[INFO] _build_inv_value_map: to inv-text: type: center
build_inv_value_map: (2/148) variable: subjectID type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (3/148) variable: followupID type: text
[INFO] _build_inv_value_map: to inv-text: type: text
build_inv_value_map: (4/148) variable: acidosis type: bool
build_inv_value_map: (5/148) variable: ageDeath_day type: int
build_inv_value_map: (6/148) variable: ageRand_hr type: float
build_inv_value_map: (7/148) variable: baselineAnticonvulsants type: bool
build_inv_value_map: (8/148) variable: dischargeAnticonvulsants type: bool
build_inv_value_map: (9/148) variable: inotropicAgent type: bool
build_inv_value_map: (10/148) variable: perinatalSentinelEvent type: bool
build_inv_value_map: (11/148) variable: dischargeSeizure type: boo

In [6]:
_INV_VALUE_MAP = {
    COMBINE_harmonizer.SHEET_MAIN: _INV_VALUE_MAP_MAIN,
    COMBINE_harmonizer.SHEET_FOLLOW_UP: _INV_VALUE_MAP_FOLLOWUP,
    COMBINE_harmonizer.SHEET_DERIVED_DATA: _INV_VALUE_MAP_DERIVED_DATA,
}

_FILENAME_INFO_INV_VALUE_MAP = {each['name']: _INV_VALUE_MAP[each['data_dict']] for each in COMBINE_harmonizer.FILENAME_INFOS}

_PREFIX_INV_VALUE_MAP = {COMBINE_harmonizer.flatten_filename_prefix(filename): val for filename, val in _FILENAME_INFO_INV_VALUE_MAP.items()}


## Load flatten

In [7]:
filename = f'{input_dir}/zz-merged-flatten.csv'

In [8]:
df = pd.read_csv(filename, dtype='O')

## 02. numerical distribution

In [9]:
list(df.columns)

['_study',
 'center',
 'subjectID',
 'uniqueID',
 '01-02:siteID',
 '01-02:birthNumber',
 '01-02:screenComment',
 '01-02:coreTempLess32p5COverEq2Hr_e',
 '01-02:coreTempLess33p5COver1Hr_e',
 '01-02:coreTempLess34COver1Hr_e',
 '01-02:first6HrCoolByClinicalProtocol_e',
 '01-02:chromosomalAbnormality_e',
 '01-02:majorCongenitalAnomaly_e',
 '01-02:birthWeightLessEq1800g_e',
 '01-02:infantUnlikelySurvive_e',
 '01-02:first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e',
 '01-02:postnatalAgeLess6HrOrGreater24Hr_e',
 '01-02:enrolledConflictingTrial_e',
 '01-02:first60MinAnyBloodGasPHLessEq7_i',
 '01-02:first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i',
 '01-02:historyPerinatalEvent_i',
 '01-02:at10MinApgarLessEq5OrVent_i',
 '01-02:randomEligible',
 '01-02:consentStatus',
 '01-02:noConsentReason',
 '01-02:noInStudyReason',
 '01-02:random',
 '01-02:noRandomReason',
 '01-02:noRandomReasonText',
 '01-02:randomDate',
 '01-02:randomTime',
 '01-02:randomNumber',
 '01-02:randomTreatmentAssi

In [10]:
columns = list(filter(lambda x: re.search('disability', x, re.I), df.columns))
columns

['30-02:disabilityLevelSurvivor',
 '30-02:disabilityLevel',
 '30-02:moderateSevereDisabilityOrDeath',
 '30-02:moderateSevereDisabilitySurvivor',
 '31-05:disabilityLevelDeath']

In [11]:
x_columns_map = {
    '01-06:cordBloodGasPH': {'title': 'Cord Blood Gas pH', 'unit': ''},

    '30-01:bloodGasPH': {'title': 'Blood Gas pH', 'unit': ''},

    '01-06:firstPostnatalBloodGasPH': {'title': 'First Postnatal Blood Gas pH', 'unit': ''},

    '01-06:Apgar1min': {'title': 'Apgar Score at 1 min', 'unit': ''},

    '01-06:Apgar5min': {'title': 'Apgar Score at 5 min', 'unit': ''},

    '01-06:Apgar10min': {'title': 'Apgar Score at 10 min', 'unit': ''},

    '31-03:MRINRNPatternOfInjuryMerge': {'title': 'NRN Pattern of Injury', 'unit': ''},
    '31-03:MRINRNPatternOfInjuryAvg': {'title': 'NRN Pattern of Injury - Mean', 'unit': ''},

    '01-06:birthGestationalAge_week': {'title': 'Birth Gestational Age', 'unit': 'Week'},

    '01-03:motherAge_year': {'title': 'Maternal Age', 'unit': 'Year'},

    '01-06:birthWeight_g': {'title': 'Birth Weight', 'unit': 'g'},

    '31-02:pre_TotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Screening', 'unit': ''},
    '31-02:post_TotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Post-treatment', 'unit': ''},
    '31-02:dischargeTotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Discharge', 'unit': ''},

    '01-06:infantSex': {'title': 'Infant Sex', 'unit': ''},
    '01-03:motherEducation': {'title': 'Maternal Education', 'unit': ''},
    '01-03:motherRace': {'title': 'Maternal Race', 'unit': ''},

    '01-05:maternalSeizure': {'title': 'Maternal Seizure', 'unit': ''},
    '01-12:pre_NeuroExamSeizure': {'title': 'Seizure - Screening', 'unit': ''},
    '03-04:post_NeuroExamSeizure': {'title': 'Seizure - Post-treatment Neuro Exam', 'unit': ''},
    '04-02:dischargeNeuroExamSeizure': {'title': 'Seizure - Discharge Neuro Exam', 'unit': ''},
    '04-13:dischargeSeizure': {'title': 'Seizure - Discharge', 'unit': ''},

    '20-04:BayleyIIICognitiveComposite': {'title': 'Bayley-III Cognitive', 'unit': ''},
    '30-02:moderateSevereDisabilityOrDeath': {'title': 'Moderate/Servere Disability or Death', 'unit': ''},
    '30-02:disabilityLevel': {'title': 'Disability Level', 'unit': ''},
}

In [12]:
def _get_order(column, value):
    prefix, var_name, postfix = COMBINE_harmonizer.flatten_column_tuple(column)
    if prefix not in _PREFIX_INV_VALUE_MAP:
        print(f'[WARN] not in _PREFIX_INV_VALUE_MAP: column: {column} prefix: {prefix}')
        return None

    inv_value_map = _PREFIX_INV_VALUE_MAP[prefix]
    if var_name not in inv_value_map:
        print(f'[WARN] not in _INV_VALUE_MAP: {var_name}')
        return None

    return inv_value_map[var_name](value)


In [13]:
def _hist(df, column, ax, row, col, info, bins=20, color='blue'):
    is_valid = df[column].isnull() == False
    value = df[column][is_valid].astype('float64')

    ax[row, col].hist(value, bins=bins, color=color)

    title = info['title']
    ax[row, col].set_title(title)


In [14]:
def _bar(df, column, ax, row, col, info, color='blue', dtype=None):
    is_valid = df[column].isnull() == False
    df_valid = df[is_valid]

    if dtype is not None:
        df_valid.loc[:, column] = df_valid[column].astype(dtype)
    df_groupby = df_valid.groupby([column]).agg({column: 'count'}).rename(columns={column: 'count'}).reset_index(drop=False)
    df_groupby['_order'] = df_groupby[column].apply(lambda x: _get_order(column, x))
    df_groupby = df_groupby.sort_values(by=['_order'], ascending=[True])

    x_value = df_groupby[column]
    y_value = df_groupby['count']

    ax[row, col].bar(x_value, y_value, color=color)

    title = info['title']
    ax[row, col].set_title(title)

In [15]:
def _pie(df, column, ax, row, col, info, color='blue', dtype=None):
    is_valid = df[column].isnull() == False
    df_valid = df[is_valid]

    is_valid_invalid = df_valid[column].isnull()
    df_valid_invalid = df_valid[is_valid_invalid]
    print(f'df_valid: column: {column} is-invalid: {len(df) - is_valid.sum()} is-null: {df_valid[column].isnull().sum()} is-valid-invalid: {is_valid_invalid.sum()} df_valid_invalid: {df_valid_invalid[column]}')

    if dtype is not None:
        df_valid.loc[:, column] = df_valid[column].astype(dtype)
    df_groupby = df_valid.groupby([column]).agg({column: 'count'}).rename(columns={column: 'count'}).reset_index(drop=False)
    df_groupby['_order'] = df_groupby[column].apply(lambda x: _get_order(column, x))
    df_groupby = df_groupby.sort_values(by=['_order'], ascending=[True])

    count_sum = df_groupby['count'].sum()
    df_groupby['ratio'] = df_groupby['count'] / count_sum

    df_groupby['x_count'] = df_groupby.apply(lambda x: f"{x[column]}\n({x['count']})", axis=1)

    x_value = list(df_groupby['x_count'])

    y_value = list(df_groupby['ratio'])

    angle = -180 * y_value[0]
    ax[row, col].pie(y_value, autopct='%1.1f%%', startangle=angle,
                         labels=x_value)

    title = info['title']
    ax[row, col].set_title(title)


In [16]:
fig, ax = plt.subplots(dpi=600, nrows=7, ncols=3, figsize=(12, 21))

column = '01-06:birthGestationalAge_week'
_bar(df, column, ax, 0, 0, x_columns_map[column], dtype='int')

column = '01-06:birthWeight_g'
_hist(df, column, ax, 0, 1, x_columns_map[column])

column = '01-06:infantSex'
_pie(df, column, ax, 0, 2, x_columns_map[column])

column = '01-03:motherAge_year'
_hist(df, column, ax, 1, 0, x_columns_map[column], bins=18)

column = '01-03:motherEducation'
_pie(df, column, ax, 1, 1, x_columns_map[column])

# column = '02-01:motherRace'
# _bar(df, column, ax, 1, 2, x_columns_map[column])
column = '01-05:maternalSeizure'
_pie(df, column, ax, 1, 2, x_columns_map[column])

column = '01-12:pre_NeuroExamSeizure'
_pie(df, column, ax, 2, 0, x_columns_map[column])

column = '03-04:post_NeuroExamSeizure'
_pie(df, column, ax, 2, 1, x_columns_map[column])

column = '04-02:dischargeNeuroExamSeizure'
_pie(df, column, ax, 2, 2, x_columns_map[column])

column = '31-02:pre_TotalModifiedSarnatScore'
_bar(df, column, ax, 3, 0, x_columns_map[column], dtype='int')

column = '31-02:post_TotalModifiedSarnatScore'
_bar(df, column, ax, 3, 1, x_columns_map[column], dtype='int')

column = '31-02:dischargeTotalModifiedSarnatScore'
_bar(df, column, ax, 3, 2, x_columns_map[column], dtype='int')

column = '31-03:MRINRNPatternOfInjuryMerge'
_bar(df, column, ax, 4, 0, x_columns_map[column])

column = '01-06:cordBloodGasPH'
_hist(df, column, ax, 4, 1, x_columns_map[column])
column = '01-06:firstPostnatalBloodGasPH'
_hist(df, column, ax, 4, 2, x_columns_map[column])

column = '01-06:Apgar1min'
_bar(df, column, ax, 5, 0, x_columns_map[column], dtype='int')
column = '01-06:Apgar5min'
_bar(df, column, ax, 5, 1, x_columns_map[column], dtype='int')
column = '01-06:Apgar10min'
_bar(df, column, ax, 5, 2, x_columns_map[column], dtype='int')

column = '20-04:BayleyIIICognitiveComposite'
_bar(df, column, ax, 6, 0, x_columns_map[column], dtype='int')

column = '30-02:moderateSevereDisabilityOrDeath'
_pie(df, column, ax, 6, 1, x_columns_map[column])

column = '30-02:disabilityLevel'
_bar(df, column, ax, 6, 2, x_columns_map[column])


plt.show()

for postfix in ['eps', 'png', 'pdf']:
    out_filename = f'{out_dir}/figure/32-distribution.{postfix}'
    fig.savefig(out_filename)

df_valid: column: 01-06:infantSex is-invalid: 0 is-null: 0 is-valid-invalid: 0 df_valid_invalid: Series([], Name: 01-06:infantSex, dtype: object)
df_valid: column: 01-03:motherEducation is-invalid: 2 is-null: 0 is-valid-invalid: 0 df_valid_invalid: Series([], Name: 01-03:motherEducation, dtype: object)
df_valid: column: 01-05:maternalSeizure is-invalid: 1 is-null: 0 is-valid-invalid: 0 df_valid_invalid: Series([], Name: 01-05:maternalSeizure, dtype: object)
df_valid: column: 01-12:pre_NeuroExamSeizure is-invalid: 0 is-null: 0 is-valid-invalid: 0 df_valid_invalid: Series([], Name: 01-12:pre_NeuroExamSeizure, dtype: object)
df_valid: column: 03-04:post_NeuroExamSeizure is-invalid: 73 is-null: 0 is-valid-invalid: 0 df_valid_invalid: Series([], Name: 03-04:post_NeuroExamSeizure, dtype: object)
df_valid: column: 04-02:dischargeNeuroExamSeizure is-invalid: 45 is-null: 0 is-valid-invalid: 0 df_valid_invalid: Series([], Name: 04-02:dischargeNeuroExamSeizure, dtype: object)


df_valid: column: 30-02:moderateSevereDisabilityOrDeath is-invalid: 28 is-null: 0 is-valid-invalid: 0 df_valid_invalid: Series([], Name: 30-02:moderateSevereDisabilityOrDeath, dtype: object)


In [17]:
df.groupby(['20-04:BayleyIIICognitiveComposite']).agg(count=('20-04:BayleyIIICognitiveComposite', 'count')).reset_index(drop=False).astype('float').sort_values(by=['20-04:BayleyIIICognitiveComposite']).reset_index(drop=True)

,20-04:BayleyIIICognitiveComposite,count
0,54.0,19.0
1,55.0,24.0
2,60.0,7.0
3,65.0,7.0
4,70.0,14.0
5,75.0,18.0
6,80.0,34.0
7,85.0,54.0
8,90.0,74.0
9,95.0,52.0
